In [ ]:
# === ECG Annotator – Tkinter prototype (uses your .txt reading logic) ===
# Requirements: pandas, numpy, matplotlib
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.gridspec import GridSpec
from datetime import timedelta
import uuid
import os

# --------------------------
# 1) Data loading (matches your notebook logic for .txt)
# --------------------------
def load_ecg_txt(
    path,
    first_seconds=10.0,
    sep=r"\t",
    decimal=",",
    dayfirst=True,
    engine="python",
):
    df = pd.read_csv(
        path,
        sep=sep,
        decimal=decimal,
        parse_dates=[[0, 1]],   # combineer datum + tijd in één kolom
        dayfirst=dayfirst,
        engine=engine
    )
    # header opschonen, hernoemen & index
    df.columns = df.columns.str.strip().str.replace(r"\s+", " ", regex=True)
    # pas kolomnaam aan naar jouw timestamp-kolomnaam
    if "yyyy-MM-dd_HH:mm:ss.ffffff" in df.columns:
        df.rename(columns={"yyyy-MM-dd_HH:mm:ss.ffffff": "timestamp"}, inplace=True)
    elif "timestamp" not in df.columns:
        # zoek heuristisch naar een samengevoegde kolom
        df.rename(columns={df.columns[0]: "timestamp"}, inplace=True)
    df.set_index("timestamp", inplace=True)

    # minimaal Lead I/II nodig, rest afleiden
    req = ["Lead I (mV)", "Lead II (mV)"]
    missing = [c for c in req if c not in df.columns]
    if missing:
        raise ValueError(f"Kolommen ontbreken: {missing}. Beschikbaar: {list(df.columns)}")

    # 6 limb leads afleiden
    df["Lead III (mV)"] = df["Lead II (mV)"] - df["Lead I (mV)"]
    df["aVR (mV)"]      = -0.5 * (df["Lead I (mV)"] + df["Lead II (mV)"])
    df["aVL (mV)"]      = df["Lead I (mV)"] - 0.5 * df["Lead II (mV)"]
    df["aVF (mV)"]      = df["Lead II (mV)"] - 0.5 * df["Lead I (mV)"]

    lead_cols = ["Lead I (mV)", "Lead II (mV)", "Lead III (mV)",
                 "aVR (mV)", "aVL (mV)", "aVF (mV)"]

    # sample rate schatten
    dts = df.index.to_series().diff().dropna().dt.total_seconds().values
    if len(dts) == 0:
        raise ValueError("Onvoldoende rijen om sampling-interval te bepalen.")
    dt = np.median(dts)
    fs = 1.0 / dt
    expected = int(round(first_seconds * fs))

    # exact first_seconds selecteren
    t0 = df.index[0]
    mask = (df.index >= t0) & (df.index < t0 + pd.Timedelta(seconds=first_seconds))
    df10 = df.loc[mask, lead_cols]
    if len(df10) > expected:
        df10 = df10.iloc[:expected]

    # tijd-as in seconden vanaf begin
    t_sec = (df10.index - df10.index[0]).total_seconds().astype(float)
    # naar numpy
    signals = {c: df10[c].to_numpy(dtype=float) for c in lead_cols}

    print(f"Schatting fs ≈ {fs:.3f} Hz | geselecteerd: {len(df10)} samples (~{expected})")
    return t_sec, signals, float(fs)

# --------------------------
# 2) Simple label model
# --------------------------
LABEL_CATEGORIES = {
    "Rhythm": ["Normal Rhythm", "Atrial Fibrillation", "Atrial Flutter", "SVT", "VT"],
    "Morphology": ["Normal", "LBBB", "RBBB", "NICD", "Ventricular"],
    "ST Changes": ["ST Elevation", "ST Depression", "QRS Widening"],
}
LABEL_COLORS = {
    "Normal Rhythm": "#4C78A8",
    "Atrial Fibrillation": "#F58518",
    "Atrial Flutter": "#E45756",
    "SVT": "#72B7B2",
    "VT": "#54A24B",
    "Normal": "#B279A2",
    "LBBB": "#FF9DA6",
    "RBBB": "#9C755F",
    "NICD": "#8C6D31",
    "Ventricular": "#EDC948",
    "ST Elevation": "#59A14F",
    "ST Depression": "#AF7AA1",
    "QRS Widening": "#F28E2B",
}

# --------------------------
# 3) Tkinter App
# --------------------------
class ECGAnnotatorApp(tk.Tk):
    def __init__(self, t, signals, fs):
        super().__init__()
        self.title("ECG Viewer (6 leads) + Extended V1 + Interval Labels")
        self.geometry("1300x900")

        self.t = t
        self.signals = signals
        self.fs = fs
        self.duration = float(self.t[-1] - self.t[0]) if self.t.size > 0 else 0.0

        self.lead_order = ["Lead I (mV)","Lead II (mV)","Lead III (mV)",
                           "aVR (mV)","aVL (mV)","aVF (mV)"]

        # annotations: list of dict {id, t0, t1, labels[list[str]]}
        self.annotations = []
        self.undo_stack = []
        self.redo_stack = []

        self._build_ui()
        self._plot_all()
        self._update_current_span()

        # ==== Zoom helpers (alleen x-as) ====
    def _init_zoom_state(self):
        # full x-range (tijd) uit data
        self._x_full = (float(self.t[0]), float(self.t[-1]))
        # actuele view (xlim) start op full
        self._x_view = list(self._x_full)
        # minimale vensterbreedte in seconden (precisie-zoom)
        self._x_min_span = 0.2

    def _apply_xlim(self, a, b):
        """Zet nieuwe x-limieten met grenzen en update ticks/redraw."""
        # respecteer grenzen en minimale span
        a = max(self._x_full[0], a)
        b = min(self._x_full[1], b)
        if b - a < self._x_min_span:
            mid = 0.5 * (a + b)
            a = mid - self._x_min_span / 2
            b = mid + self._x_min_span / 2
            a = max(self._x_full[0], a)
            b = min(self._x_full[1], b)
        self._x_view = [a, b]
        self.ax_avg.set_xlim(a, b)
        # ticks opnieuw (past zich aan aan zoom)
        self._set_halfsec_ticks(self.ax_avg)
        # houd current selection zichtbaar
        self._update_current_span()
        self.canvas.draw_idle()

    def _reset_zoom(self):
        self._apply_xlim(*self._x_full)

    # ==== Helpers: tijd-snapping, tick-helpers, cursor-state ====
    def _snap(self, t: float) -> float:
        if t is None:
            return None
        # clamp naar plot-range en snap op 0.1 s
        t = max(self.t[0], min(self.t[-1], t))
        return round(t * 10) / 10.0

    def _set_halfsec_ticks(self, ax):
        """
        Major ticks elke 1.0 s met labels; minor elke 0.5 s (zonder labels).
        Dit oogt rustig en is consistent over alle plots.
        """
        ax.xaxis.set_major_locator(MultipleLocator(1.0))
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.xaxis.set_minor_locator(MultipleLocator(0.5))
        ax.grid(True, which='major', alpha=0.25)
        ax.grid(True, which='minor', alpha=0.10)

    def _init_cursor(self):
        # rood stipje + verticale lijn
        self._cursor_dot, = self.ax_avg.plot([], [], 'o', color='#CC3333', ms=6, zorder=10)
        self._cursor_vline = self.ax_avg.axvline(self.t[0], color='#CC3333', alpha=0.35, lw=1.0)
        self._dragging = False
        self._t_anchor = None
        # overlay voor huidig interval (één artist die we updaten)
        self._current_span = None

        # ==== Mouse interaction on Extended V1 plot ====
    def _on_mouse_move(self, event):
        if event.inaxes != self.ax_avg or event.xdata is None or event.ydata is None:
            return
        t = self._snap(event.xdata)
        # cursor visuals
        self._cursor_dot.set_data([t], [event.ydata])
        self._cursor_vline.set_xdata([t, t])
        # toon selectie live als we slepen
        if self._dragging and self._t_anchor is not None:
            a = self._snap(min(self._t_anchor, t))
            b = self._snap(max(self._t_anchor, t))
            if b <= a:
                b = a + 0.1
            # sliders laten meebewegen
            self.var_start.set(a)
            self.var_end.set(b)
            self._update_current_span()
        else:
            # alleen cursor updaten
            self.canvas.draw_idle()
        # statuslabel updaten
        self.lbl_sel.config(text=f"Selected: {self.var_start.get():.1f}–{self.var_end.get():.1f} s | Cursor: {t:.1f}s")

    def _on_mouse_press(self, event):
        # Alleen linker muisknop op V1-avg
        if event.button != 1 or event.inaxes != self.ax_avg or event.xdata is None:
            return
        t = self._snap(event.xdata)
        self._dragging = True
        self._t_anchor = t
        # start vastklikken; init klein interval zodat feedback zichtbaar is
        self.var_start.set(t)
        self.var_end.set(self._snap(t + 0.1))
        self._update_current_span()

    def _on_mouse_release(self, event):
        if not self._dragging:
            return
        self._dragging = False
        if event.inaxes != self.ax_avg or event.xdata is None:
            # muis buiten plot losgelaten -> niets wijzigen
            return
        t = self._snap(event.xdata)
        a = self._snap(min(self._t_anchor, t))
        b = self._snap(max(self._t_anchor, t))
        if b <= a:
            b = a + 0.1
        self.var_start.set(a)
        self.var_end.set(b)
        self._update_current_span()
        # anker loslaten
        self._t_anchor = None

    def _on_scroll(self, event):
        """Zoom in/uit op de V1-plot met het muiswiel, gecentreerd op cursor."""
        if event.inaxes != self.ax_avg or event.xdata is None:
            return

        # richting bepalen (Matplotlib varieert per backend)
        step = getattr(event, "step", None)
        if step is None:
            # fallback op button: 'up' = inzoomen, 'down' = uitzoomen
            direction = -1 if event.button == 'down' else 1
        else:
            direction = 1 if step > 0 else -1

        # zoom factor (per 'tick')
        zoom_base = 1.2
        factor = (zoom_base if direction > 0 else 1.0/zoom_base)

        x0, x1 = self._x_view
        span = (x1 - x0)
        if span <= 0:
            return

        # zoom rond cursor-x
        cx = float(event.xdata)
        new_span = span / factor
        # behoud cursor-punt relatief binnen het venster
        rel = (cx - x0) / span
        a = cx - rel * new_span
        b = a + new_span

        self._apply_xlim(a, b)

    def _on_mouse_press(self, event):
        # rechter muisknop = reset zoom
        if event.inaxes == self.ax_avg and event.button == 3:
            self._reset_zoom()
            return

        # Alleen linker muisknop op V1-avg voor selectie
        if event.button != 1 or event.inaxes != self.ax_avg or event.xdata is None:
            return
        t = self._snap(event.xdata)
        self._dragging = True
        self._t_anchor = t
        self.var_start.set(t)
        self.var_end.set(self._snap(t + 0.1))
        self._update_current_span()

    # ---- UI layout
    def _build_ui(self):
        root = ttk.Frame(self)
        root.pack(fill="both", expand=True)

        left = ttk.Frame(root)
        left.pack(side="left", fill="both", expand=True)
        right = ttk.Frame(root, width=320)
        right.pack(side="right", fill="y")

        # Matplotlib figure (2x3 + extended V1)
        self.fig = plt.Figure(figsize=(9, 8), dpi=100)
        gs = GridSpec(4, 2, height_ratios=[1,1,1,1.2], figure=self.fig)
        self.axes = {}
        for i, lead in enumerate(self.lead_order):
            r, c = divmod(i, 2)
            ax = self.fig.add_subplot(gs[r, c])
            ax.set_title(lead)
            ax.set_ylabel("mV")
            ax.grid(True, alpha=0.2)
            self.axes[lead] = ax
        self.ax_avg = self.fig.add_subplot(gs[3, :])
        self.ax_avg.set_title("Extended V1 (Average of All Leads)")
        self.ax_avg.set_xlabel("Time (s)")
        self.ax_avg.set_ylabel("mV")
        self.ax_avg.grid(True, alpha=0.2)

        self.canvas = FigureCanvasTkAgg(self.fig, master=left)

        # --- Muisinteractie op de V1-avg plot ---
        self._mpl_cids = []
        self._mpl_cids.append(self.canvas.mpl_connect('motion_notify_event', self._on_mouse_move))
        self._mpl_cids.append(self.canvas.mpl_connect('button_press_event', self._on_mouse_press))
        self._mpl_cids.append(self.canvas.mpl_connect('button_release_event', self._on_mouse_release))
        self._mpl_cids.append(self.canvas.mpl_connect('scroll_event', self._on_scroll))

        self.canvas.draw()
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

        # Controls on right
        box_interval = ttk.LabelFrame(right, text="Interval Selection (0.1 s)")
        box_interval.pack(fill="x", padx=8, pady=8)

        self.var_start = tk.DoubleVar(value=2.0)
        self.var_end   = tk.DoubleVar(value=2.5)

        self.scale_start = ttk.Scale(box_interval, from_=0.0, to=max(0.1, self.t[-1]),
                                     orient="horizontal", variable=self.var_start,
                                     command=lambda v: self._on_scale_change("start"))
        self.scale_end = ttk.Scale(box_interval, from_=0.1, to=max(0.2, self.t[-1]),
                                   orient="horizontal", variable=self.var_end,
                                   command=lambda v: self._on_scale_change("end"))

        ttk.Label(box_interval, text="Start (s)").pack(anchor="w", padx=6, pady=(6,0))
        self.scale_start.pack(fill="x", padx=6)
        ttk.Label(box_interval, text="End (s)").pack(anchor="w", padx=6, pady=(6,0))
        self.scale_end.pack(fill="x", padx=6)
        self.lbl_sel = ttk.Label(box_interval, text="Selected: 2.0–2.5 s")
        self.lbl_sel.pack(anchor="w", padx=6, pady=6)

        box_label = ttk.LabelFrame(right, text="Add Label")
        box_label.pack(fill="x", padx=8, pady=8)
        self.combo_cat = ttk.Combobox(box_label, values=list(LABEL_CATEGORIES.keys()), state="readonly")
        self.combo_cat.set("Rhythm")
        self.combo_cat.pack(fill="x", padx=6, pady=(6,3))
        self.combo_label = ttk.Combobox(box_label, values=LABEL_CATEGORIES["Rhythm"], state="readonly")
        self.combo_label.set(LABEL_CATEGORIES["Rhythm"][0])
        self.combo_label.pack(fill="x", padx=6, pady=3)

        btn_add = ttk.Button(box_label, text="Add to interval", command=self._add_label_to_interval)
        btn_add.pack(fill="x", padx=6, pady=(6,6))

        self.combo_cat.bind("<<ComboboxSelected>>", self._on_cat_change)

        box_actions = ttk.LabelFrame(right, text="Actions")
        box_actions.pack(fill="x", padx=8, pady=8)
        ttk.Button(box_actions, text="New Interval Here", command=self._ensure_current_interval).pack(fill="x", padx=6, pady=4)
        ttk.Button(box_actions, text="Undo", command=self._undo).pack(fill="x", padx=6, pady=4)
        ttk.Button(box_actions, text="Redo", command=self._redo).pack(fill="x", padx=6, pady=4)
        ttk.Button(box_actions, text="Export CSV", command=self._export_csv).pack(fill="x", padx=6, pady=4)

        box_list = ttk.LabelFrame(right, text="Annotated Intervals")
        box_list.pack(fill="both", expand=True, padx=8, pady=8)
        cols = ("t0","t1","labels")
        self.tree = ttk.Treeview(box_list, columns=cols, show="headings", height=12)
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=90 if c!="labels" else 160, anchor="w")
        self.tree.pack(fill="both", expand=True, padx=6, pady=6)

        btn_del = ttk.Button(box_list, text="Delete selected", command=self._delete_selected)
        btn_del.pack(fill="x", padx=6, pady=(0,6))

    # ---- plotting
    def _plot_all(self):
        # 6 leads
        for i, lead in enumerate(self.lead_order):
            ax = self.axes[lead]
            ax.clear()
            ax.set_title(lead)
            ax.set_ylabel("mV")
            ax.plot(self.t, self.signals[lead], linewidth=1.0, color="#444")
            self._set_halfsec_ticks(ax)
            if i < len(self.lead_order) - 1:  # x-as labels alleen onderaan de grid via V1-avg
                ax.set_xticklabels([])

        # Extended V1 avg
        self.ax_avg.clear()
        self.ax_avg.set_title("Extended V1 (Average of All Leads)")
        self.ax_avg.set_xlabel("Time (s)")
        self.ax_avg.set_ylabel("mV")
        avg = np.mean([self.signals[k] for k in self.lead_order], axis=0)
        self.ax_avg.plot(self.t, avg, linewidth=1.8, color="#AA3333")
        self._set_halfsec_ticks(self.ax_avg)

        # bestaand opgeslagen intervallen als vlakken (achtergrond)
        for it in self.annotations:
            self._draw_interval_span(self.ax_avg, it["t0"], it["t1"], it["labels"])

        # (her)initialiseer cursor + leeg current-span
        self._init_cursor()

        # (her)initialiseer cursor + leeg current-span
        self._init_cursor()

        # (her)initialiseer zoom state en zet full view
        self._init_zoom_state()
        self._apply_xlim(*self._x_full)

        self.canvas.draw_idle()

    def _draw_interval_span(self, ax, t0, t1, labels):
        # kleur op basis van eerste label (of default)
        color = LABEL_COLORS.get(labels[0], "#5B5B5B") if labels else "#5B5B5B"
        ax.axvspan(t0, t1, alpha=0.18, color=color)

    # ---- interval selection sliders
    def _on_scale_change(self, which):
        # snap naar 0.1 s
        if which == "start":
            v = round(self.var_start.get()*10)/10.0
            self.var_start.set(v)
            if self.var_end.get() <= v:
                self.var_end.set(round((v+0.1)*10)/10.0)
        else:
            v = round(self.var_end.get()*10)/10.0
            self.var_end.set(v)
            if v <= self.var_start.get():
                self.var_start.set(round((v-0.1)*10)/10.0)
        self._update_current_span()

    def _current_bounds(self):
        t0 = round(self.var_start.get()*10)/10.0
        t1 = round(self.var_end.get()*10)/10.0
        if t1 <= t0: t1 = t0 + 0.1
        return float(t0), float(t1)

    def _update_current_span(self):
        a, b = self._current_bounds()
        # verwijder vorige current-span indien aanwezig
        if self._current_span is not None:
            try:
                self._current_span.remove()
            except Exception:
                pass
            self._current_span = None
        # teken nieuwe current-span
        self._current_span = self.ax_avg.axvspan(a, b, alpha=0.25, color="#3399FF", zorder=5)
        self.lbl_sel.config(text=f"Selected: {a:.1f}–{b:.1f} s")
        self.canvas.draw_idle()

    # ---- annotations
    def _ensure_current_interval(self):
        t0, t1 = self._current_bounds()
        # bestaat exact? -> return id, anders aanmaken
        for it in self.annotations:
            if abs(it["t0"]-t0)<1e-9 and abs(it["t1"]-t1)<1e-9:
                return it["id"]
        new_it = {"id": str(uuid.uuid4()), "t0": t0, "t1": t1, "labels": []}
        self.annotations.append(new_it)
        self.undo_stack.append(("del", (new_it,)))
        self.redo_stack.clear()
        self._refresh_list()
        self._update_current_span()
        return new_it["id"]

    def _on_cat_change(self, _):
        cat = self.combo_cat.get()
        self.combo_label["values"] = LABEL_CATEGORIES[cat]
        self.combo_label.set(LABEL_CATEGORIES[cat][0])

    def _add_label_to_interval(self):
        label = self.combo_label.get().strip()
        if not label:
            return
        iid = self._ensure_current_interval()
        it = next(x for x in self.annotations if x["id"] == iid)
        if label not in it["labels"]:
            it["labels"].append(label)
            self.undo_stack.append(("remove_label", (iid, label)))
            self.redo_stack.clear()
        self._refresh_list()
        self._update_current_span()

    def _delete_selected(self):
        sel = self.tree.selection()
        if not sel: return
        iid = self.tree.item(sel[0], "values")[0]  # we stoppen id in hidden? -> zet id als eerste col
        # in deze implementatie zetten we id niet als zichtbare kolom; pak via tag opslaan:
        # eenvoudiger: zoek op t0,t1; daarom slaan we id in 'text' of in iid map – hier eenvoud:
        # we zoeken best matching interval via t0,t1 uit tree row:
        vals = self.tree.item(sel[0], "values")
        t0 = float(vals[0]); t1 = float(vals[1])
        for k, it in enumerate(self.annotations):
            if abs(it["t0"]-t0)<1e-9 and abs(it["t1"]-t1)<1e-9:
                removed = self.annotations.pop(k)
                self.undo_stack.append(("add_back", (removed,)))
                self.redo_stack.clear()
                break
        self._refresh_list()
        self._update_current_span()

    def _undo(self):
        if not self.undo_stack: return
        cmd, args = self.undo_stack.pop()
        self._apply(cmd, args, self.redo_stack)
        self._refresh_list()
        self._update_current_span()

    def _redo(self):
        if not self.redo_stack: return
        cmd, args = self.redo_stack.pop()
        self._apply(cmd, args, self.undo_stack)
        self._refresh_list()
        self._update_current_span()

    def _apply(self, cmd, args, push_stack):
        if cmd == "del":
            (it,) = args
            # remove it
            for k, cur in enumerate(self.annotations):
                if cur["id"] == it["id"]:
                    removed = self.annotations.pop(k)
                    push_stack.append(("add_back", (removed,)))
                    return
        elif cmd == "add_back":
            (it,) = args
            self.annotations.append(it)
            push_stack.append(("del", (it,)))
        elif cmd == "remove_label":
            iid, label = args
            for it in self.annotations:
                if it["id"] == iid and label in it["labels"]:
                    it["labels"].remove(label)
                    push_stack.append(("add_label", (iid, label)))
                    return
        elif cmd == "add_label":
            iid, label = args
            for it in self.annotations:
                if it["id"] == iid and label not in it["labels"]:
                    it["labels"].append(label)
                    push_stack.append(("remove_label", (iid, label)))
                    return

    def _refresh_list(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        # sorteer op t0
        for it in sorted(self.annotations, key=lambda x: (x["t0"], x["t1"])):
            lbl = ", ".join(it["labels"]) if it["labels"] else "(no labels)"
            self.tree.insert("", "end", values=(f"{it['t0']:.1f}", f"{it['t1']:.1f}", lbl))

    # ---- export
    def _export_csv(self):
        if not self.annotations:
            messagebox.showinfo("Export CSV", "Geen intervallen om te exporteren.")
            return
        path = filedialog.asksaveasfilename(
            title="Export labels to CSV",
            defaultextension=".csv",
            initialfile="labels_export.csv",
            filetypes=[("CSV", "*.csv")]
        )
        if not path:
            return
        rows = []
        for it in self.annotations:
            rows.append({
                "t_start": it["t0"],
                "t_end": it["t1"],
                "lead": "V1-avg",
                "labels": ";".join(it["labels"])
            })
        pd.DataFrame(rows).to_csv(path, index=False)
        messagebox.showinfo("Export CSV", f"Exported {len(rows)} intervals to:\n{os.path.abspath(path)}")

# ---- SAFE RUN BLOCK (vervang je huidige "Run" sectie hiermee) ----
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

# 1) Gebruik één verborgen Tk-root voor dialogs (en hergebruik die tussen runs)
if 'TK_ROOT' not in globals():
    TK_ROOT = tk.Tk()
    TK_ROOT.withdraw()
    TK_ROOT.call('wm', 'attributes', '.', '-topmost', True)   # voorkom dat dialoog achter andere vensters zit
    TK_ROOT.call('wm', 'attributes', '.', '-topmost', False)

# 2) Sluit een eventuele oude app + figuren netjes
def _cleanup_previous():
    global app
    try:
        app.destroy()
    except Exception:
        pass
    plt.close('all')

_cleanup_previous()

# 3) Toon dialoog, maar breek schoon af bij Annuleren
path_txt = filedialog.askopenfilename(
    parent=TK_ROOT,
    title="Select ECG .txt",
    filetypes=[("Text files","*.txt"),("All files","*.*")]
)

if not path_txt:
    print("Annuleren gekozen. Geen bestand geladen. (Run deze cel opnieuw om opnieuw te proberen.)")
else:
    # Laad data en start app
    try:
        t_sec, signals, fs = load_ecg_txt(path_txt, first_seconds=10.0)
    except Exception as e:
        print(f"Fout bij inladen: {e}")
    else:
        app = ECGAnnotatorApp(t=t_sec, signals=signals, fs=fs)
        # Zorg dat sluiten van het venster ook resources vrijgeeft
        app.protocol("WM_DELETE_WINDOW", app.destroy)
        app.mainloop()

C:\Users\MSI\AppData\Local\Temp\ipykernel_145628\3155381932.py:28: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
C:\Users\MSI\AppData\Local\Temp\ipykernel_145628\3155381932.py:28: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S.%f format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df = pd.read_csv(


Schatting fs ≈ 500.250 Hz | geselecteerd: 4877 samples (~5003)


KeyboardInterrupt: 